In [ ]:
!pip install geopandas
!pip install jenkspy
!pip install matplotlib
!pip install pandas
!pip install numpy

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import jenkspy
import time
from shapely.geometry import Point
import plotly.graph_objs as go
import csv

# ------------------------------------------
# Função: Gerar dados de teste simulados
# ------------------------------------------
def gerar_dados_simulados(n=500):
    lats = -30.0 + 0.1 * np.random.rand(n)
    lons = -51.0 + 0.1 * np.random.rand(n)
    valores = np.random.gamma(shape=2.0, scale=100.0, size=n)
    df = pd.DataFrame({
        'latitude': lats,
        'longitude': lons,
        'pop_density': valores
    })
    geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
    return gdf

# ------------------------------------------
# Função GADF
# ------------------------------------------
def calcular_gadf(valores, quebras):
    media_total = np.mean(valores)
    desvio_total = np.sum(np.abs(valores - media_total))
    desvio_intra = 0
    for i in range(len(quebras) - 1):
        grupo = valores[(valores >= quebras[i]) & (valores <= quebras[i+1])]
        if len(grupo) > 0:
            media_grupo = np.mean(grupo)
            desvio_intra += np.sum(np.abs(grupo - media_grupo))
    return 1 - (desvio_intra / desvio_total) if desvio_total != 0 else 0

# ------------------------------------------
# Classificação Jenks com retorno de info
# ------------------------------------------
def classificar_jenks(gdf, coluna_numerica='pop_density'):
    gdf = gdf.dropna(subset=[coluna_numerica])
    valores = gdf[coluna_numerica].values
    valores_ordenados = np.sort(valores)

    resultados = []
    for k in range(2, 7):  # Máximo de 6 classes
        quebras = jenkspy.jenks_breaks(valores_ordenados, n_classes=k)
        gadf = calcular_gadf(valores_ordenados, quebras)
        resultados.append((k, gadf, quebras))

    k_otimo, gadf_otimo, quebras_finais = max(resultados, key=lambda x: x[1])
    gdf["classe"] = pd.cut(gdf[coluna_numerica], bins=quebras_finais, labels=False, include_lowest=True)

    return gdf, k_otimo, gadf_otimo, quebras_finais

# ------------------------------------------
# Executar pipeline incremental
# ------------------------------------------
def executar_pipeline_incremental(gdf_total, ciclo, historico_gadf):
    novos_dados = gerar_dados_simulados()
    gdf_total = pd.concat([gdf_total, novos_dados], ignore_index=True)

    gdf_total, k_otimo, gadf, quebras = classificar_jenks(gdf_total)

    print(f"\n--- Atualização {ciclo + 1} ---")
    print(f"Número ótimo de classes: {k_otimo}")
    print(f"GADF: {gadf:.4f}")
    print("Intervalos por classe:")
    for i in range(len(quebras) - 1):
        print(f"Classe {i}: {quebras[i]:.2f} a {quebras[i+1]:.2f}")

    # Atualiza o histórico
    historico_gadf.append(gadf)

    # Salvar gráfico interativo em HTML com Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=list(range(1, len(historico_gadf) + 1)),
        y=historico_gadf,
        mode='lines+markers',
        marker=dict(size=8),
        hovertemplate='Ciclo %{x}<br>GADF: %{y:.4f}<extra></extra>'
    ))
    fig.update_layout(
        title='Evolução do Índice GADF por Ciclo',
        xaxis_title='Ciclo',
        yaxis_title='GADF',
        yaxis=dict(range=[0, 1]),
        template='plotly_white'
    )
    fig.write_html(f"grafico_gadf_interativo_ciclo_{ciclo + 1}.html")
    fig.show()

    # Salvar histórico em CSV
    with open("historico_gadf.csv", "a", newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        if ciclo == 0:
            writer.writerow(["ciclo", "valores_classes", "gadf"])
        intervalos_str = "; ".join([f"{quebras[i]:.2f}-{quebras[i+1]:.2f}" for i in range(len(quebras) - 1)])
        writer.writerow([ciclo + 1, intervalos_str, f"{gadf:.4f}"])

    return gdf_total, historico_gadf

# ------------------------------------------
# Loop principal com dados acumulados
# ------------------------------------------
gdf_acumulado = gpd.GeoDataFrame(columns=['latitude', 'longitude', 'pop_density', 'geometry'], geometry='geometry', crs="EPSG:4326")
historico_gadf = []

for i in range(10000):  # Você pode aumentar para mais ciclos
    gdf_acumulado, historico_gadf = executar_pipeline_incremental(gdf_acumulado, i, historico_gadf)
    time.sleep(20)  # Ajuste para 30 segundos se quiser



--- Atualização 1 ---
Número ótimo de classes: 6
GADF: 0.7854
Intervalos por classe:
Classe 0: 7.51 a 89.17
Classe 1: 89.17 a 162.95
Classe 2: 162.95 a 247.63
Classe 3: 247.63 a 349.95
Classe 4: 349.95 a 479.18
Classe 5: 479.18 a 821.23


<ipython-input-12-3ec38fe41914>:65: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.




--- Atualização 2 ---
Número ótimo de classes: 6
GADF: 0.7593
Intervalos por classe:
Classe 0: 7.51 a 97.06
Classe 1: 97.06 a 180.95
Classe 2: 180.95 a 281.14
Classe 3: 281.14 a 414.04
Classe 4: 414.04 a 608.19
Classe 5: 608.19 a 960.24



--- Atualização 3 ---
Número ótimo de classes: 6
GADF: 0.7716
Intervalos por classe:
Classe 0: 7.51 a 96.25
Classe 1: 96.25 a 177.99
Classe 2: 177.99 a 270.18
Classe 3: 270.18 a 386.23
Classe 4: 386.23 a 559.28
Classe 5: 559.28 a 1175.53



--- Atualização 4 ---
Número ótimo de classes: 6
GADF: 0.7642
Intervalos por classe:
Classe 0: 7.51 a 98.15
Classe 1: 98.15 a 182.86
Classe 2: 182.86 a 281.14
Classe 3: 281.14 a 412.61
Classe 4: 412.61 a 621.64
Classe 5: 621.64 a 1205.50


...


--- Atualização 251 ---
Número ótimo de classes: 6
GADF: 0.7612
Intervalos por classe:
Classe 0: 0.31 a 103.65
Classe 1: 103.65 a 189.32
Classe 2: 189.32 a 287.61
Classe 3: 287.61 a 413.17
Classe 4: 413.17 a 600.92
Classe 5: 600.92 a 1538.37



--- Atualização 252 ---
Número ótimo de classes: 6
GADF: 0.7612
Intervalos por classe:
Classe 0: 0.31 a 103.65
Classe 1: 103.65 a 189.34
Classe 2: 189.34 a 287.62
Classe 3: 287.62 a 413.24
Classe 4: 413.24 a 600.92
Classe 5: 600.92 a 1538.37



--- Atualização 253 ---
Número ótimo de classes: 6
GADF: 0.7612
Intervalos por classe:
Classe 0: 0.31 a 103.64
Classe 1: 103.64 a 189.32
Classe 2: 189.32 a 287.62
Classe 3: 287.62 a 413.21
Classe 4: 413.21 a 600.92
Classe 5: 600.92 a 1538.37



--- Atualização 254 ---
Número ótimo de classes: 6
GADF: 0.7611
Intervalos por classe:
Classe 0: 0.31 a 103.65
Classe 1: 103.65 a 189.34
Classe 2: 189.34 a 287.62
Classe 3: 287.62 a 413.19
Classe 4: 413.19 a 600.92
Classe 5: 600.92 a 1538.37


KeyboardInterrupt: 

In [ ]:
import os
import random
import time
import zipfile
import io
import csv
from typing import Tuple, Dict

import folium
from folium.plugins import MarkerCluster
# Importar branca.colormap para criar os mapas de cores
import branca.colormap as cm
import geopandas as gpd
import numpy as np
import pandas as pd
import requests
from shapely.geometry import Point, LineString, Polygon
import jenkspy
import plotly.graph_objects as go
from google.colab import files


# Diretórios temporários para salvar arquivos
os.makedirs("saida", exist_ok=True)
os.makedirs("saida_geometria_rs", exist_ok=True) # Criar o diretório para os mapas de geometria

# --- Funções de carregamento e classificação (mantidas do seu código original) ---
def carregar_dados(caminho: str = None, via_web: bool = True) -> pd.DataFrame:
    if via_web:
        try:
            url_raw = caminho.replace("github.com", "raw.githubusercontent.com").replace("/blob/", "/")
            response = requests.get(url_raw)
            response.raise_for_status()
            return pd.read_csv(io.StringIO(response.text))
        except Exception as e:
            raise Exception(f"Erro ao carregar arquivo da web: {e}")
    else:
        uploaded = files.upload()
        for filename in uploaded:
            return pd.read_csv(io.StringIO(uploaded[filename].decode('utf-8')))

def is_quantitativo(serie: pd.Series) -> bool:
    return pd.api.types.is_numeric_dtype(serie)

def classificar_subtipo(serie: pd.Series) -> str:
    serie = serie.dropna()
    valores_unicos = serie.unique()
    n_unique = len(valores_unicos)
    if n_unique == 1:
        return "constante"
    elif all(float(x).is_integer() for x in valores_unicos):
        return "ordinal" if n_unique <= 10 else "discreto"
    else:
        return "contínuo"

def classificar_colunas(df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, str]]:
    classificacoes = {}
    subtipos = {}
    for col in df.columns:
        serie = df[col]
        if is_quantitativo(serie):
            classificacoes[col] = "quantitativo"
            subtipos[col] = classificar_subtipo(serie)
        else:
            classificacoes[col] = "qualitativo"
    df_quanti = df[[col for col, tipo in classificacoes.items() if tipo == "quantitativo"]].copy()
    linha_subtipos = pd.DataFrame([subtipos], index=["_subtipo"])
    df_final = pd.concat([linha_subtipos, df_quanti], axis=0)
    return df_final, classificacoes

def salvar_arquivos_temporarios(df_quanti: pd.DataFrame, classificacoes: Dict[str, str], ciclo: int = None):
    # Salva arquivos temporários que serão incluídos no ZIP final
    suffix = f"_{ciclo}" if ciclo is not None else ""
    caminho_csv = f"saida/dados_limpos_quanti{suffix}.csv"
    caminho_html = f"saida/dados_limpos_quanti{suffix}.html"
    caminho_txt = f"saida/classificacao_colunas{suffix}.txt"

    df_quanti.to_csv(caminho_csv, index=False, encoding="utf-8-sig")
    df_quanti.to_html(caminho_html, index=False)

    with open(caminho_txt, "w", encoding="utf-8") as f:
        for col, tipo in classificacoes.items():
            f.write(f"{col}: {tipo}\n")

    print(f"Arquivos de dados e classificação gerados para o ciclo {ciclo}")

# --- Funções GADF + Jenks (mantidas do seu código original) ---
def calcular_gadf(valores, quebras):
    media_total = np.mean(valores)
    desvio_total = np.sum(np.abs(valores - media_total))
    desvio_intra = 0
    for i in range(len(quebras) - 1):
        grupo = valores[(valores >= quebras[i]) & (valores <= quebras[i+1])]
        if len(grupo) > 0:
            media_grupo = np.mean(grupo)
            desvio_intra += np.sum(np.abs(grupo - media_grupo))
    return 1 - (desvio_intra / desvio_total) if desvio_total != 0 else 0

def classificar_jenks(gdf, coluna_numerica='pop_density'):
    gdf = gdf.dropna(subset=[coluna_numerica])
    if gdf.empty:
        return gdf, 0, 0, []
    valores = gdf[coluna_numerica].values
    valores_ordenados = np.sort(valores)

    # Evita erro se todos os valores são iguais ou há menos de 2 valores únicos para Jenks
    if len(np.unique(valores_ordenados)) < 2:
        # Se todos os valores são iguais, atribua uma única classe
        gdf["classe"] = 0
        return gdf, 1, 0, [valores_ordenados[0], valores_ordenados[0] + 0.1] # Retorna uma classe única e bins válidos

    resultados = []
    # Garante que o número de classes não exceda o número de valores únicos
    max_classes = min(6, len(np.unique(valores_ordenados)))
    if max_classes < 2:
      # Se não há classes suficientes para Jenks, retorna com uma classe
      gdf["classe"] = 0
      return gdf, 1, 0, [valores_ordenados[0], valores_ordenados[0] + 0.1]


    for k in range(2, max_classes + 1):
        try:
            quebras = jenkspy.jenks_breaks(valores_ordenados, n_classes=k)
            # Remove quebras duplicadas que jenkspy pode gerar (raramente, mas acontece)
            quebras_unicas = sorted(list(set(quebras)))
            # Se após dropar duplicatas, o número de classes for muito baixo
            if len(quebras_unicas) < 2:
                 continue # Tenta com um número diferente de classes

            gadf = calcular_gadf(valores_ordenados, quebras_unicas)
            resultados.append((k, gadf, quebras_unicas))
        except ValueError as e:
            print(f"Erro ao calcular Jenks para {k} classes: {e}. Detalhes: {e}")
            continue

    if not resultados:
        # Se nenhuma classificação Jenks foi bem-sucedida, atribua uma classe padrão
        gdf["classe"] = 0
        return gdf, 0, 0, []

    k_otimo, gadf_otimo, quebras_finais = max(resultados, key=lambda x: x[1])

    # AQUI ESTÁ A MUDANÇA: Adiciona duplicates='drop'
    gdf["classe"] = pd.cut(gdf[coluna_numerica], bins=quebras_finais, labels=False, include_lowest=True, duplicates='drop')

    return gdf, k_otimo, gadf_otimo, quebras_finais

# --- Geração de dados RS aleatórios (com acúmulo) ---
def gerar_dados_geo_aleatorios(n=50, existing_df=None):
    if existing_df is None:
        dados_novos = []
        start_id = 1
    else:
        # Aumentar o ID para garantir unicidade com os dados anteriores
        start_id = existing_df['id'].max() + 1 if not existing_df.empty else 1
        dados_novos = existing_df.to_dict('records') # Converte o DataFrame existente para lista de dicionários

    for i in range(start_id, start_id + n):
        tipo_geometria = random.choice(['Ponto', 'Linha', 'Polígono'])
        lat_min, lat_max = -33.75, -27.5
        lon_min, lon_max = -57.5, -49.5

        data_row = {'id': i, 'nome': f'Elemento {i}', 'classe': random.randint(1, 5)}

        if tipo_geometria == 'Ponto':
            data_row['lat'] = round(random.uniform(lat_min, lat_max), 6)
            data_row['lon'] = round(random.uniform(lon_min, lon_max), 6)
            data_row['geometry_type'] = 'Point'

        elif tipo_geometria == 'Linha':
            lat1 = round(random.uniform(lat_min, lat_max), 6)
            lon1 = round(random.uniform(lon_min, lon_max), 6)
            lat2 = round(random.uniform(lat_min, lat_max), 6)
            lon2 = round(random.uniform(lon_min, lon_max), 6)
            data_row['coordenadas_linha'] = [(lon1, lat1), (lon2, lat2)]
            data_row['geometry_type'] = 'LineString'

        elif tipo_geometria == 'Polígono':
            lats = [round(random.uniform(lat_min, lat_max), 6) for _ in range(3)]
            lons = [round(random.uniform(lon_min, lon_max), 6) for _ in range(3)]
            # Fecha o polígono
            data_row['coordenadas_poligono'] = [(lons[0], lats[0]), (lons[1], lats[1]), (lons[2], lats[2]), (lons[0], lats[0])]
            data_row['geometry_type'] = 'Polygon'

        # Adicionar um valor numérico para a coluna 'pop_density' ou similar para Jenks
        data_row['pop_density'] = random.uniform(10, 1000)
        # Adicionar um 'radius' para as áreas simuladas, se necessário no futuro
        data_row['radius'] = random.uniform(0.1, 0.7)


        dados_novos.append(data_row)

    # Criar um DataFrame a partir da lista combinada de dicionários
    df_acumulado = pd.DataFrame(dados_novos)
    return df_acumulado

# --- Funções de criação e salvamento de mapas (com layouts mantidos) ---

# Paleta de cores Oranges_09 em formato hexadecimal
ORANGES_09_COLORS = ['#FFF5EB', '#FEE6CE', '#FDD0A2', '#FDAE6B', '#FD8D3C', '#F16913', '#D94801', '#A63603', '#7F2704']

# Função para criar um mapa de pontos
def criar_mapa_pontos(gdf_pontos, ciclo):
    estado = "RS"
    titulo_mapa = f"Layout de Mapas Temático Quantitativos de Pontos (Ciclo {ciclo})"

    mapa = folium.Map(location=[-30.0, -53.0], zoom_start=6, tiles="OpenStreetMap")

    pontos = folium.FeatureGroup(name="Pontos (Classe)")

    if not gdf_pontos.empty and 'classe' in gdf_pontos.columns and pd.api.types.is_numeric_dtype(gdf_pontos['classe']):
        min_classe = gdf_pontos['classe'].min()
        max_classe = gdf_pontos['classe'].max()
        if min_classe == max_classe: # Para evitar erro se houver apenas uma classe
            # Define o intervalo para o mapa de cores se houver apenas uma classe
            cores = cm.LinearColormap(ORANGES_09_COLORS, vmin=min_classe, vmax=min_classe + 1)
        else:
            cores = cm.LinearColormap(ORANGES_09_COLORS, vmin=min_classe, vmax=max_classe)
    else:
        # Default se não houver dados ou classe válida
        cores = cm.LinearColormap(ORANGES_09_COLORS, vmin=1, vmax=5)


    for _, row in gdf_pontos.iterrows():
        # Certifique-se de que 'classe' é um número
        classe = int(row['classe']) if pd.isna(row['classe']) == False else 1 # Default se for NaN
        cor = cores(classe)
        tamanho = 4 + classe * 2
        coords = [row.geometry.y, row.geometry.x]

        folium.CircleMarker(
            location=coords,
            radius=tamanho,
            color=cor,
            fill=True,
            fill_color=cor,
            fill_opacity=0.8,
            popup=f"{row['nome']} - Classe: {classe}"
        ).add_to(pontos)

    cores.caption = "Classe"
    cores.add_to(mapa)
    pontos.add_to(mapa)

    # Camadas de estado e municípios
    url_estado = f"https://raw.githubusercontent.com/josecacciatore89/Estados_BR_IBGE/main/{estado}.geojson"
    camada_estado = folium.FeatureGroup(name=f"Estado - {estado}")
    try:
        geojson_estado = requests.get(url_estado).json()
        folium.GeoJson(
            geojson_estado,
            name=estado,
            style_function=lambda x: {
                "fillOpacity": 0.0,
                "color": "black",
                "weight": 2
            },
            highlight_function=lambda x: {"weight": 3, "color": "blue"},
            tooltip=estado
        ).add_to(camada_estado)
        camada_estado.add_to(mapa)
    except Exception as e:
        print(f"Erro ao carregar estado {estado}: {e}")

    url_municipios = f"https://raw.githubusercontent.com/josecacciatore89/municipios_BR_IBGE2023/main/{estado}_2023.geojson"
    camada_municipios = folium.FeatureGroup(name=f"Municípios {estado}")
    try:
        geojson_municipios = requests.get(url_municipios).json()
        folium.GeoJson(
            geojson_municipios,
            name=f"Municipios_{estado}",
            style_function=lambda x: {
                "fillOpacity": 0.0,
                "color": "black",
                "weight": 0.5
            },
            popup=folium.GeoJsonPopup(fields=["NM_MUN"]),
            tooltip=folium.GeoJsonTooltip(fields=["NM_MUN"])
        ).add_to(camada_municipios)
        camada_municipios.add_to(mapa)
    except Exception as e:
        print(f"Erro ao carregar municípios do estado {estado}: {e}")

    folium.LayerControl(collapsed=False).add_to(mapa)

    cor_fundo = "#FFFFFF"
    cor_fonte = "#000000"
    html = f"""
    <div style="background-color: {cor_fundo}; color: {cor_fonte}; font-family: Arial; padding: 20px; text-align: center; font-weight: bold; font-size: 20px;">
        {titulo_mapa}
    </div>
    """
    mapa.get_root().html.add_child(folium.Element(html))

    left_logo = '<img src="https://raw.githubusercontent.com/josecacciatore89/imagens/main/cropped-logo.png" style="position:absolute;top:10px;left:100px;width:150px;">'
    right_logo = '<img src="https://raw.githubusercontent.com/josecacciatore89/imagens/main/cropped-logo.png" style="position:absolute;top:10px;right:100px;width:150px;">'
    mapa.get_root().html.add_child(folium.Element(left_logo))
    mapa.get_root().html.add_child(folium.Element(right_logo))

    mapa.save(f"saida_geometria_rs/mapa_pontos_{ciclo}.html")
    print(f"Mapa de pontos salvo como saida_geometria_rs/mapa_pontos_{ciclo}.html")
    return mapa

# Função para criar um mapa de linhas
def criar_mapa_linhas(gdf_linhas, ciclo):
    estado = "RS"
    titulo_mapa = f"Layout de Mapas Temático Quantitativos de Linhas (Ciclo {ciclo})"

    mapa = folium.Map(location=[-30.0, -53.0], zoom_start=6, tiles="OpenStreetMap")

    linhas = folium.FeatureGroup(name="Linhas (Classe)")
    if not gdf_linhas.empty and 'classe' in gdf_linhas.columns and pd.api.types.is_numeric_dtype(gdf_linhas['classe']):
        min_classe = gdf_linhas['classe'].min()
        max_classe = gdf_linhas['classe'].max()
        if min_classe == max_classe:
            cores = cm.LinearColormap(ORANGES_09_COLORS, vmin=min_classe, vmax=min_classe + 1)
        else:
            cores = cm.LinearColormap(ORANGES_09_COLORS, vmin=min_classe, vmax=max_classe)
    else:
        cores = cm.LinearColormap(ORANGES_09_COLORS, vmin=1, vmax=5)

    for _, row in gdf_linhas.iterrows():
        classe = int(row['classe']) if pd.isna(row['classe']) == False else 1
        cor = cores(classe)
        espessura = 1 + classe * 2
        coords = [(lat, lon) for lon, lat in row.geometry.coords]

        folium.PolyLine(
            locations=coords,
            color=cor,
            weight=espessura,
            popup=f"{row['nome']} - Classe: {classe}"
        ).add_to(linhas)

    cores.caption = "Classe"
    cores.add_to(mapa)
    linhas.add_to(mapa)

    # Camadas de estado e municípios (repetição do código anterior)
    url_estado = f"https://raw.githubusercontent.com/josecacciatore89/Estados_BR_IBGE/main/{estado}.geojson"
    camada_estado = folium.FeatureGroup(name=f"Estado - {estado}")
    try:
        geojson_estado = requests.get(url_estado).json()
        folium.GeoJson(
            geojson_estado,
            name=estado,
            style_function=lambda x: {
                "fillOpacity": 0.0,
                "color": "black",
                "weight": 2
            },
            highlight_function=lambda x: {"weight": 3, "color": "blue"},
            tooltip=estado
        ).add_to(camada_estado)
        camada_estado.add_to(mapa)
    except Exception as e:
        print(f"Erro ao carregar estado {estado}: {e}")

    url_municipios = f"https://raw.githubusercontent.com/josecacciatore89/municipios_BR_IBGE2023/main/{estado}_2023.geojson"
    camada_municipios = folium.FeatureGroup(name=f"Municípios {estado}")
    try:
        geojson_municipios = requests.get(url_municipios).json()
        folium.GeoJson(
            geojson_municipios,
            name=f"Municipios_{estado}",
            style_function=lambda x: {
                "fillOpacity": 0.0,
                "color": "black",
                "weight": 0.5
            },
            popup=folium.GeoJsonPopup(fields=["NM_MUN"]),
            tooltip=folium.GeoJsonTooltip(fields=["NM_MUN"])
        ).add_to(camada_municipios)
        camada_municipios.add_to(mapa)
    except Exception as e:
        print(f"Erro ao carregar municípios do estado {estado}: {e}")

    folium.LayerControl(collapsed=False).add_to(mapa)

    cor_fundo = "#FFFFFF"
    cor_fonte = "#000000"
    html = f"""
    <div style="background-color: {cor_fundo}; color: {cor_fonte}; font-family: Arial; padding: 20px; text-align: center; font-weight: bold; font-size: 20px;">
        {titulo_mapa}
    </div>
    """
    mapa.get_root().html.add_child(folium.Element(html))

    left_logo = '<img src="https://raw.githubusercontent.com/josecacciatore89/imagens/main/cropped-logo.png" style="position:absolute;top:10px;left:100px;width:150px;">'
    right_logo = '<img src="https://raw.githubusercontent.com/josecacciatore89/imagens/main/cropped-logo.png" style="position:absolute;top:10px;right:100px;width:150px;">'
    mapa.get_root().html.add_child(folium.Element(left_logo))
    mapa.get_root().html.add_child(folium.Element(right_logo))

    mapa.save(f"saida_geometria_rs/mapa_linhas_{ciclo}.html")
    print(f"Mapa de linhas salvo como saida_geometria_rs/mapa_linhas_{ciclo}.html")
    return mapa

# Função para criar um mapa de polígonos/áreas
def criar_mapa_poligonos(gdf_poligonos, ciclo):
    estado = "RS"
    titulo_mapa = f"Layout de Mapas Temático Quantitativos de Áreas (Ciclo {ciclo})"

    mapa = folium.Map(location=[-30.0, -53.0], zoom_start=6, tiles="OpenStreetMap")

    areas = folium.FeatureGroup(name="Áreas (Classe)")
    if not gdf_poligonos.empty and 'classe' in gdf_poligonos.columns and pd.api.types.is_numeric_dtype(gdf_poligonos['classe']):
        min_classe = gdf_poligonos['classe'].min()
        max_classe = gdf_poligonos['classe'].max()
        if min_classe == max_classe:
            cores = cm.LinearColormap(ORANGES_09_COLORS, vmin=min_classe, vmax=min_classe + 1)
        else:
            cores = cm.LinearColormap(ORANGES_09_COLORS, vmin=min_classe, vmax=max_classe)
    else:
        cores = cm.LinearColormap(ORANGES_09_COLORS, vmin=1, vmax=5)

    # Função para calcular um deslocamento para evitar sobreposição (se aplicável para simulação)
    def desloca_area(lat, lon, deslocamento_lat, deslocamento_lon):
        return [lat + deslocamento_lat, lon + deslocamento_lon]

    # Variáveis para o deslocamento (apenas para simulação, se não for de geometria real)
    deslocamento_lat_simulado = 0.01
    deslocamento_lon_simulado = 0.01

    for _, row in gdf_poligonos.iterrows():
        classe = int(row['classe']) if pd.isna(row['classe']) == False else 1
        cor = cores(classe)

        # Se a geometria for um Polígono real do GeoDataFrame
        if isinstance(row.geometry, Polygon):
            coords = [(lat, lon) for lon, lat in row.geometry.exterior.coords]
            folium.Polygon(
                locations=coords,
                color=cor,
                fill=True,
                fill_color=cor,
                fill_opacity=0.6,
                popup=f"{row['nome']} - Classe: {classe}"
            ).add_to(areas)
        # Se for uma "Área" simulada (do seu código original, usando ponto central + raio)
        # Note: Esta parte pode gerar polígonos sobrepostos se os dados aleatórios forem muito próximos
        elif 'lat' in row and 'lon' in row and 'radius' in row:
            coords = [row.geometry.y, row.geometry.x]
            tamanho = row['radius'] # Tamanho do lado das áreas (em graus)
            new_coords = desloca_area(coords[0], coords[1], deslocamento_lat_simulado, deslocamento_lon_simulado)
            area_coords = [
                [new_coords[0] + tamanho, new_coords[1] - tamanho],
                [new_coords[0] + tamanho, new_coords[1] + tamanho],
                [new_coords[0] - tamanho, new_coords[1] + tamanho],
                [new_coords[0] - tamanho, new_coords[1] - tamanho]
            ]
            folium.Polygon(
                locations=area_coords,
                color=cor,
                fill=True,
                fill_color=cor,
                fill_opacity=0.6,
                popup=f"{row['nome']} - Classe: {classe}"
            ).add_to(areas)


    cores.caption = "Classe"
    cores.add_to(mapa)
    areas.add_to(mapa)

    # Camadas de estado e municípios (repetição do código anterior)
    url_estado = f"https://raw.githubusercontent.com/josecacciatore89/Estados_BR_IBGE/main/{estado}.geojson"
    camada_estado = folium.FeatureGroup(name=f"Estado - {estado}")
    try:
        geojson_estado = requests.get(url_estado).json()
        folium.GeoJson(
            geojson_estado,
            name=estado,
            style_function=lambda x: {
                "fillOpacity": 0.0,
                "color": "black",
                "weight": 2
            },
            highlight_function=lambda x: {"weight": 3, "color": "blue"},
            tooltip=estado
        ).add_to(camada_estado)
        camada_estado.add_to(mapa)
    except Exception as e:
        print(f"Erro ao carregar estado {estado}: {e}")

    url_municipios = f"https://raw.githubusercontent.com/josecacciatore89/municipios_BR_IBGE2023/main/{estado}_2023.geojson"
    camada_municipios = folium.FeatureGroup(name=f"Municípios {estado}")
    try:
        geojson_municipios = requests.get(url_municipios).json()
        folium.GeoJson(
            geojson_municipios,
            name=f"Municipios_{estado}",
            style_function=lambda x: {
                "fillOpacity": 0.0,
                "color": "black",
                "weight": 0.5
            },
            popup=folium.GeoJsonPopup(fields=["NM_MUN"]),
            tooltip=folium.GeoJsonTooltip(fields=["NM_MUN"])
        ).add_to(camada_municipios)
        camada_municipios.add_to(mapa)
    except Exception as e:
        print(f"Erro ao carregar municípios do estado {estado}: {e}")

    folium.LayerControl(collapsed=False).add_to(mapa)

    cor_fundo = "#FFFFFF"
    cor_fonte = "#000000"
    html = f"""
    <div style="background-color: {cor_fundo}; color: {cor_fonte}; font-family: Arial; padding: 20px; text-align: center; font-weight: bold; font-size: 20px;">
        {titulo_mapa}
    </div>
    """
    mapa.get_root().html.add_child(folium.Element(html))

    left_logo = '<img src="https://raw.githubusercontent.com/josecacciatore89/imagens/main/cropped-logo.png" style="position:absolute;top:10px;left:100px;width:150px;">'
    right_logo = '<img src="https://raw.githubusercontent.com/josecacciatore89/imagens/main/cropped-logo.png" style="position:absolute;top:10px;right:100px;width:150px;">'
    mapa.get_root().html.add_child(folium.Element(left_logo))
    mapa.get_root().html.add_child(folium.Element(right_logo))

    mapa.save(f"saida_geometria_rs/mapa_areas_deslocadas_{ciclo}.html")
    print(f"Mapa de áreas salvo como saida_geometria_rs/mapa_areas_deslocadas_{ciclo}.html")
    return mapa

# --- Função Principal para Orquestração ---
def pipeline_atualizacao_geo(num_ciclos: int = 5, intervalo_segundos: int = 20):
    print("Iniciando a pipeline de atualização de dados e mapas...")

    all_data_df = pd.DataFrame() # DataFrame para acumular todos os dados gerados
    historico_gadf_global = []

    for ciclo in range(num_ciclos):
        print(f"\n--- Iniciando Ciclo {ciclo + 1} de {num_ciclos} ---")

        # 1. Geração de novos dados aleatórios e acumulação
        # Gerar mais dados aleatórios a cada ciclo
        novos_dados_df = gerar_dados_geo_aleatorios(n=50) # Adiciona 50 novos registros a cada ciclo
        all_data_df = pd.concat([all_data_df, novos_dados_df], ignore_index=True)
        print(f"Total de registros acumulados: {len(all_data_df)}")

        # 2. Classificação das colunas (mantida do seu código)
        df_quanti, classificacoes = classificar_colunas(all_data_df)
        salvar_arquivos_temporarios(df_quanti, classificacoes, ciclo) # Salva os arquivos classificados

        # 3. Preparação dos GeoDataFrames para os mapas
        # Pontos
        gdf_pontos = all_data_df[all_data_df['geometry_type'] == 'Point'].copy()
        if not gdf_pontos.empty:
            gdf_pontos['geometry'] = gdf_pontos.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
            gdf_pontos = gpd.GeoDataFrame(gdf_pontos, geometry='geometry', crs="EPSG:4326")
            gdf_pontos, k_otimo_pontos, gadf_pontos, quebras_pontos = classificar_jenks(gdf_pontos, 'pop_density')
            print(f"Jenks para Pontos: k_otimo={k_otimo_pontos}, GADF={gadf_pontos:.4f}")
            criar_mapa_pontos(gdf_pontos, ciclo)
        else:
            print("Nenhum dado de ponto para gerar mapa.")

        # Linhas
        gdf_linhas = all_data_df[all_data_df['geometry_type'] == 'LineString'].copy()
        if not gdf_linhas.empty:
            gdf_linhas['geometry'] = gdf_linhas['coordenadas_linha'].apply(lambda coords: LineString(coords))
            gdf_linhas = gpd.GeoDataFrame(gdf_linhas, geometry='geometry', crs="EPSG:4326")
            gdf_linhas, k_otimo_linhas, gadf_linhas, quebras_linhas = classificar_jenks(gdf_linhas, 'pop_density')
            print(f"Jenks para Linhas: k_otimo={k_otimo_linhas}, GADF={gadf_linhas:.4f}")
            criar_mapa_linhas(gdf_linhas, ciclo)
        else:
            print("Nenhum dado de linha para gerar mapa.")

        # Polígonos
        gdf_poligonos = all_data_df[all_data_df['geometry_type'] == 'Polygon'].copy()
        if not gdf_poligonos.empty:
            gdf_poligonos['geometry'] = gdf_poligonos['coordenadas_poligono'].apply(lambda coords: Polygon(coords))
            gdf_poligonos = gpd.GeoDataFrame(gdf_poligonos, geometry='geometry', crs="EPSG:4326")
            gdf_poligonos, k_otimo_poligonos, gadf_poligonos, quebras_poligonos = classificar_jenks(gdf_poligonos, 'pop_density')
            print(f"Jenks para Polígonos: k_otimo={k_otimo_poligonos}, GADF={gadf_poligonos:.4f}")
            criar_mapa_poligonos(gdf_poligonos, ciclo)
        else:
            print("Nenhum dado de polígono para gerar mapa.")

        # Acompanhamento do GADF acumulado
        if 'pop_density' in all_data_df.columns and not all_data_df.empty:
            temp_gdf_for_gadf = all_data_df.copy()
            if 'geometry' not in temp_gdf_for_gadf.columns and 'lon' in temp_gdf_for_gadf.columns and 'lat' in temp_gdf_for_gadf.columns:
                temp_gdf_for_gadf = temp_gdf_for_gadf.dropna(subset=['lon', 'lat'])
                if not temp_gdf_for_gadf.empty:
                    temp_gdf_for_gadf['geometry'] = gpd.points_from_xy(temp_gdf_for_gadf['lon'], temp_gdf_for_gadf['lat'])

            if not temp_gdf_for_gadf.empty and 'geometry' in temp_gdf_for_gadf.columns:
                temp_gdf_for_gadf = gpd.GeoDataFrame(temp_gdf_for_gadf, geometry='geometry', crs="EPSG:4326")
                # Classifica Jenks para a coluna 'pop_density' para calcular GADF
                temp_gdf_for_gadf, k_otimo_temp, gadf_temp, quebras_temp = classificar_jenks(temp_gdf_for_gadf, 'pop_density')
                historico_gadf_global.append(gadf_temp)

                # Salvar o gráfico de GADF
                fig_gadf = go.Figure()
                fig_gadf.add_trace(go.Scatter(x=list(range(1, len(historico_gadf_global)+1)), y=historico_gadf_global, mode='lines+markers'))
                fig_gadf.update_layout(title='Evolução do GADF (Dados Acumulados)', xaxis_title='Ciclo', yaxis_title='GADF', template='plotly_white')
                fig_gadf.write_html(f"saida/grafico_gadf_acumulado_ciclo_{ciclo + 1}.html")
                print(f"Gráfico GADF acumulado salvo como saida/grafico_gadf_acumulado_ciclo_{ciclo + 1}.html")
            else:
                print("Dados insuficientes para calcular GADF acumulado neste ciclo.")
        else:
            print("Coluna 'pop_density' não encontrada ou dados insuficientes para calcular GADF acumulado.")

        if ciclo < num_ciclos - 1: # Não espera no último ciclo
            print(f"Aguardando {intervalo_segundos} segundos para o próximo ciclo...")
            time.sleep(intervalo_segundos)

    print("\n--- Pipeline de atualização concluída! ---")
    print("Gerando arquivo ZIP final com todos os resultados de todos os ciclos...")

    final_zip_path = "todos_os_resultados_geoespaciais.zip"

    with zipfile.ZipFile(final_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Adiciona arquivos do diretório 'saida'
        for root, _, files_in_dir in os.walk('saida'):
            for file in files_in_dir:
                file_path = os.path.join(root, file)
                # O arcname mantém a estrutura de diretórios 'saida/...' dentro do ZIP
                arcname = os.path.relpath(file_path, os.getcwd())
                zipf.write(file_path, arcname=arcname)

        # Adiciona arquivos do diretório 'saida_geometria_rs'
        for root, _, files_in_dir in os.walk('saida_geometria_rs'):
            for file in files_in_dir:
                file_path = os.path.join(root, file)
                # O arcname mantém a estrutura de diretórios 'saida_geometria_rs/...' dentro do ZIP
                arcname = os.path.relpath(file_path, os.getcwd())
                zipf.write(file_path, arcname=arcname)

    print(f"Todos os resultados foram compactados em: {final_zip_path}")
    # Automaticamente baixa o arquivo ZIP no Google Colab
    files.download(final_zip_path)

# --- Execução da pipeline ---
if __name__ == "__main__":
    # Remove o arquivo historico_gadf.csv se existir para começar limpo
    # (Este arquivo é sobrescrito a cada execução, mas a verificação é boa)
    if os.path.exists("saida/historico_gadf.csv"):
        os.remove("saida/historico_gadf.csv")

    # Executar a pipeline. Você pode ajustar num_ciclos e intervalo_segundos aqui:
    # Exemplo: 10 ciclos, com 10 segundos entre cada ciclo
    pipeline_atualizacao_geo(num_ciclos=10, intervalo_segundos=10)

Iniciando a pipeline de atualização de dados e mapas...

--- Iniciando Ciclo 1 de 10 ---
Total de registros acumulados: 50
Arquivos de dados e classificação gerados para o ciclo 0
Jenks para Pontos: k_otimo=6, GADF=0.6483
Mapa de pontos salvo como saida_geometria_rs/mapa_pontos_0.html
Jenks para Linhas: k_otimo=6, GADF=0.7047
Mapa de linhas salvo como saida_geometria_rs/mapa_linhas_0.html
Jenks para Polígonos: k_otimo=6, GADF=0.7574
Mapa de áreas salvo como saida_geometria_rs/mapa_areas_deslocadas_0.html
Gráfico GADF acumulado salvo como saida/grafico_gadf_acumulado_ciclo_1.html
Aguardando 10 segundos para o próximo ciclo...

--- Iniciando Ciclo 2 de 10 ---
Total de registros acumulados: 100
Arquivos de dados e classificação gerados para o ciclo 1
Jenks para Pontos: k_otimo=6, GADF=0.7729
Mapa de pontos salvo como saida_geometria_rs/mapa_pontos_1.html
Jenks para Linhas: k_otimo=6, GADF=0.7748
Mapa de linhas salvo como saida_geometria_rs/mapa_linhas_1.html
Jenks para Polígonos: k_otimo=

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>